In [2]:
# importamos las librerías necesarias
import sim          # librería para conectar con CoppeliaSim
import sympy as sp  # librería para cálculo simbólico
import numpy as np

In [ ]:
# Comenzaremos el trabajo desde la descripción de Denavit-Hartenberg para el robot
#      theta     |      d      |      a      |    alpha
# ---------------------------------------------------------
#      q1        |    0.302    |    0.467    |     0
#      q2        |    -0.01    |    0.4005   |    180
#      0         |      q3     |     0       |     0
#      q4        |      0      |     0       |    180
# ---------------------------------------------------------
#      0         |    -0.058   |     0       |     0

In [ ]:
# Definimos una función para construir las matrices de transformación
# en forma simbóĺica a partir de los parámetros D-H

def symTfromDH(theta, d, a, alpha):
    # theta y alpha en radianes
    # d y a en metros
    Rz = sp.Matrix([[sp.cos(theta), -sp.sin(theta), 0, 0],
                   [sp.sin(theta), sp.cos(theta), 0, 0],
                   [0, 0, 1, 0],
                   [0, 0, 0, 1]])
    tz = sp.Matrix([[1, 0, 0, 0],
                   [0, 1, 0, 0],
                   [0, 0, 1, d],
                   [0, 0, 0, 1]])
    ta = sp.Matrix([[1, 0, 0, a],
                   [0, 1, 0, 0],
                   [0, 0, 1, 0],
                   [0, 0, 0, 1]])
    Rx = sp.Matrix([[1, 0, 0, 0],
                   [0, sp.cos(alpha), -sp.sin(alpha), 0],
                   [0, sp.sin(alpha), sp.cos(alpha), 0],
                   [0, 0, 0, 1]])
    T = Rz*tz*ta*Rx
    return T

In [ ]:
L3 = 150
L4 = 100
L5 = 50

l1 = 150 # (0,300)
l2 = 50 # (0,200)
q1 = sp.pi # (0,2pi)
q2 = sp.pi/2 # (-pi/2 , pi/2)

theta = [sp.pi/2,-sp.pi/2, 0 , q1 ,q2]
d = [L3,l1,l2,0,0]
a = [0,0,0,L4,L5]
alpha = [sp.pi/2,-sp.pi/2,0,0,0]

T01 = symTfromDH(theta[0],d[0],a[0],alpha[0])
T12 = symTfromDH(theta[1],d[1],a[1],alpha[1])
T23 = symTfromDH(theta[2],d[2],a[2],alpha[2])
T34 = symTfromDH(theta[3],d[3],a[3],alpha[3])
T45 = symTfromDH(theta[4],d[4],a[4],alpha[4])

T05 = T01*T12*T23*T34*T45
print(T05)

In [ ]:
# Definimos un punto de destino
x = 0.5
y = 0.5
z = 0.1

In [ ]:
# preparamos las ecuaciones transformando las expresiones
# de la forma <expresion = valor> a la forma <expresion - valor> = 0


In [ ]:
def connect(port):
# Establece la conexión a VREP
# port debe coincidir con el puerto de conexión en VREP
# retorna el número de cliente o -1 si no puede establecer conexión
    sim.simxFinish(-1) # just in case, close all opened connections
    clientID=sim.simxStart('127.0.0.1',port,True,True,2000,5) # Conectarse
    if clientID == 0: print("conectado a", port)
    else: print("no se pudo conectar")
    return clientID

In [88]:
# Requerimos los manejadores para las articulaciones
clientID = connect(25000)

retCode,pjoint0=sim.simxGetObjectHandle(clientID,'pjoint0',sim.simx_opmode_blocking) 
retCode,pjoint1=sim.simxGetObjectHandle(clientID,'pjoint1',sim.simx_opmode_blocking) 
retCode,rjoint0=sim.simxGetObjectHandle(clientID,'rjoint0',sim.simx_opmode_blocking) 
retCode,rjoint1=sim.simxGetObjectHandle(clientID,'rjoint1',sim.simx_opmode_blocking) 
retCode,end_effector=sim.simxGetObjectHandle(clientID,'EndEffector',sim.simx_opmode_blocking) 

# Lectura posicion del efector final



conectado a 25000


In [93]:
# Enviamos las posiciones a las articulaciones
l1 = 1
l2 = 1
q1 = 0*sp.pi/180
q2 = 90*sp.pi/180
retCode = sim.simxSetJointTargetPosition(clientID,pjoint0,l1, sim.simx_opmode_oneshot)
retCode = sim.simxSetJointTargetPosition(clientID,pjoint1,l2, sim.simx_opmode_oneshot)
retCode = sim.simxSetJointTargetPosition(clientID,rjoint0,q1, sim.simx_opmode_oneshot)
retCode = sim.simxSetJointTargetPosition(clientID,rjoint1,q2, sim.simx_opmode_oneshot)
print(retCode) # 0 Ok

returnCode,ef_pos = sim.simxGetObjectPosition(clientID,end_effector,-1,sim.simx_opmode_blocking) # Read tip position relative to base(-1) in meters
print([ef_pos])

0
[[0.5047889947891235, 1.0000088214874268, 0.4954685568809509]]
